In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
from scipy import stats
import os
from IPython.display import clear_output
import rasterio as rio
from rasterio.enums import Resampling
from rasterio.profiles import DefaultGTiffProfile

def CreateFolder(directory):
    try :
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory.' + directory)

name1     = 'co2'
name2     = 'cog'

CreateFolder('./'+str(name2)+'_down_sampled')

path1     = './'+str(name1)
path2     = './'+str(name2)

cond      = 10

path_list1 = sorted([f for f in os.listdir(str(path1)) if f.endswith('.tif')])
path_list2 = sorted([f for f in os.listdir(str(path2)) if f.endswith('.tif')])
ds         = rio.open(str(path1)+'/'+str(path_list1[0]))
d_shape    = (ds.shape[0],ds.shape[1],2,len(path_list1))
corr0      = -3000*np.ones((d_shape) ,dtype=np.float32)
corr       = -3000*np.ones((ds.shape),dtype=np.float32)
sp_cor     = np.zeros((len(path_list1),3))
ds.close()

for i,file1 in enumerate(path_list1):
    ds1 = rio.open(str(path1)+'/'+str(file1))
    ds2 = rio.open(str(path2)+'/'+str(path_list2[i]))
    
    upscale_factor = 1/10
    data2 = ds2.read(out_shape=(ds2.count,int(ds2.height*upscale_factor),int(ds2.width*upscale_factor)),
                    resampling=Resampling.average)
    transform = ds2.transform*ds2.transform.scale((ds2.width / data2.shape[-1]),(ds2.height / data2.shape[-2]))
    
    ind = np.array(np.where(data2[0]>-10))
    x,y = rio.transform.xy(transform,ind[0],ind[1],offset='center')
    xy  = np.array([x,y])

    row,col = rio.transform.rowcol(ds1.transform,xy[0],xy[1])
    rc      = np.array([row,col])

    kwds  = ds1.profile
    kwds['nodata'] = -3000.0
    
    ds3   = rio.open('./'+str(name2)+'_down_sampled/'+str(path_list2[i]), 'w', **kwds)
    s     = -3000*np.ones((ds1.shape),dtype=np.float32)
    data1 = ds1.read(1)
    ind1  = np.array(np.where(data1>-10)).T
    for j in range(ind1.shape[0]):
        A = (rc[0]==ind1[j,0])&(rc[1]==ind1[j,1])
        if len(A[A==True])>0:
            s[ind1[j,0],ind1[j,1]] = np.mean(data2[0,ind[0,A],ind[1,A]])
            corr0[ind1[j,0],ind1[j,1],0,i] = s[ind1[j,0],ind1[j,1]]
            corr0[ind1[j,0],ind1[j,1],1,i] = data1[ind1[j,0],ind1[j,1]]
                 
    ds3.write(s,1)
    ind2 = np.array((data1>-10)&(s>-10))
    A    = stats.spearmanr(data1[ind2], s[ind2])
    sp_cor[i,0] = i
    sp_cor[i,1] = A[0]
    sp_cor[i,2] = A[1]
    ds1.close()
    ds2.close()
    ds3.close()
    
np.savetxt('./'+str(name1)+'_'+str(name2)+'_Spatial_correlation.txt',sp_cor,fmt='     '.join(['%i']+['%1.2f']*2)
           ,header = 'No   Spearman_Corr   Significante ')

np.save('./'+str(name2)+'_corr0.npy', corr0)

ModuleNotFoundError: No module named 'rasterio'